In [ ]:
# ===============================================================
# Author  : Jiayao Yu, User Interfaces Group, Aalto University
# Init    : August, 2017
# Project : ELEC-D7010 Engineering for Humans course materials
# Topic   : Keystoke-level Model (KLM)
# ===============================================================

# ===============================================================
# Usage description
# -------------------
# - Middle Google GUI: afford searching task performance
# - Modelling panel: record operator sequence, count and calculate 
# total task completion time
# - If report problems on picture loading when running the notebook,
# plase go to menu 'Kernel' -> 'Restart&Clear Output' -> run again
# ===============================================================

from tkinter import *
from tkinter import ttk
from PIL import ImageTk, Image
import sys, os
import webbrowser
from collections import Counter

#define operators
global K
K = 0
global B
B = 0
global P
P = 0
global H
H = 0
global D
D = 0
global M
M = 0
#flags for later operator sequence logic
global flagFirstClick
flagFirstClick=True
global flagContinuousClick
flagContinuousClick=False
global flagContinuouInput
flagContinuouInput=False
global counter
counter=0
global counterClick
counterClick=0
global counterInput
counterInput=0

root=Tk()
root.title("KLM GUI: Search Engine Example")
content=ttk.Frame(root,padding=(10,10,10,10))
content.grid(column=0,row=0,sticky=(N,S,E,W))

#user task and KLM intro UI
explanationFrame=ttk.Labelframe(content,text='User Task')
explanation=Message(explanationFrame,text="In the google search enging, please input a string in query field, then click on search button / press <ENTER> key",padx=10,pady=10)
frame=Frame(content)
label=Label(content,text="placeholder")
wikipediaButton=Button(content,text="KLM on Wikipedia",height=1)
explanationFrame.grid(column=0,row=0,rowspan=4,sticky=(N,S,E,W),padx=10,pady=10)
explanation.grid(column=0,row=0,sticky=(E,S,W,N))
frame.grid(column=0,row=4,rowspan=7,sticky=(E,S,W,N),padx=10,pady=10)
label.grid(in_=frame)
label.lower(frame)
wikipediaButton.grid(column=0,row=11,sticky=(E,S,W,N),padx=10,pady=10)

#Google search GUI
ui=Frame(content,relief=GROOVE,borderwidth=2)
ui.config(background="white")
logoPath="google_logo.jpg"
logoPic=ImageTk.PhotoImage(Image.open(logoPath).resize((200,108)))
logoLabel=Label(ui,image=logoPic)
logoLabel.config(background="white")
logoLabel.image=logoPic
queryField=Entry(ui,width=70)
searchPicPath="search.png"
searchPic=ImageTk.PhotoImage(Image.open(searchPicPath).resize((20,20)))
searchButton=Button(ui,text="search",image=searchPic,bg='#4c92f1',width=20,relief=FLAT)
searchButton.image=searchPic
cover=Frame(ui,background="white")
resultPicPath="result_scribble.png"
resultPic=ImageTk.PhotoImage(Image.open(resultPicPath).resize((300,150)))
placeholder=Label(ui,image=resultPic,background="white",anchor=CENTER)
placeholder.image=resultPic
ui.grid(column=1,row=0,rowspan=12,sticky=(E,S,W,N),padx=10,pady=10)
logoLabel.grid(row=0,column=0,columnspan=6,sticky=(E,W),padx=10)
queryField.grid(row=1,column=0,columnspan=5,sticky=(E,S,W,N),padx=(30,0),pady=10)
searchButton.grid(row=1,column=5,sticky=(E,S,W,N),padx=(0,30),pady=10)
cover.grid(row=2,column=0,columnspan=6,sticky=(E,S,W,N),padx=(30,30),pady=30)
placeholder.grid(in_=cover,sticky=(S,N),padx=(70,0))
placeholder.lower(cover)

modelFrame=ttk.Labelframe(content,text='Modelling')
#operator sequence display
scrollbar=Scrollbar(modelFrame,width=5)
operatorSequenceTitle=Label(modelFrame,text="Operator Sequence:")
operatorSequence=Text(modelFrame,height=10,width=10,padx=10,pady=10,yscrollcommand=scrollbar.set)
operatorSequenceTmp=operatorSequence.get("1.0", 'end-1c')
scrollbar.config(command=operatorSequence.yview)
#operator count display
operatorCountTitle=Label(modelFrame,text="Operator Count:")
operatorK=Label(modelFrame,text="K =")
operatorB=Label(modelFrame,text="B =")
operatorP=Label(modelFrame,text="P =")
operatorH=Label(modelFrame,text="H =")
operatorD=Label(modelFrame,text="D =")
operatorM=Label(modelFrame,text="M =")
operatorR=Label(modelFrame,text="R =")
K=IntVar()
K.set(0)
operatorKCount=Label(modelFrame,textvariable=K)
B=IntVar()
B.set(0)
operatorBCount=Label(modelFrame,textvariable=B)
P=IntVar()
P.set(0)
operatorPCount=Label(modelFrame,textvariable=P)
H=IntVar()
H.set(0)
operatorHCount=Label(modelFrame,textvariable=H)
D=IntVar()
D.set(0)
operatorDCount=Label(modelFrame,textvariable=D)
M=IntVar()
M.set(0)
operatorMCount=Label(modelFrame,textvariable=M)
R=IntVar()
R.set(0)
operatorRCount=Label(modelFrame,textvariable=R)
operatorRCount2=Label(modelFrame,textvariable=R)
#operator values display
operatorValuesTitle=Label(modelFrame,text="Operator Values (second):")
operatorValuesK=Label(modelFrame,text="Keystroke\t= 0.20",underline=(0))
operatorValuesB=Label(modelFrame,text="Button press\t= 0.10",underline=(0))
operatorValuesP=Label(modelFrame,text="Pointing\t\t= 1.10",underline=(0))
operatorValuesH=Label(modelFrame,text="Homing\t\t= 0.40",underline=(0))
operatorValuesD=Label(modelFrame,text="Drawing\t\t= .9n+.16l",underline=(0))
operatorValuesM=Label(modelFrame,text="Mentally preparing = 1.35",underline=(0))
operatorValuesR=Label(modelFrame,text="Response from system = depends",underline=(0))
#reset button to restart modelling
resetButton=Button(modelFrame,text="Reset",width=15,bg='#4c92f1',fg="white")
#total time display
totalTimeTitle=Label(modelFrame,text="Total Time (second):")
time=DoubleVar()
time.set(0)
totalTime=Label(modelFrame,textvariable=time)
totalTimePlus=Label(modelFrame,text="+")
totalTimeR=Label(modelFrame,text="R")
#modelling UI layout
modelFrame.grid(row=0,column=2,rowspan=12,sticky=(E,S,W,N),padx=10,pady=10)
operatorSequenceTitle.grid(row=0,column=0,columnspan=24,sticky=(W),padx=10)
operatorSequence.grid(row=1,column=0,columnspan=23,sticky=(E,S,W,N),padx=10,pady=10)
scrollbar.grid(row=1,column=23,sticky=(E,S,W,N),padx=10,pady=10)
operatorCountTitle.grid(row=2,column=0,columnspan=12,sticky=(W,S,N),padx=10,pady=10)
operatorValuesTitle.grid(row=2,column=12,columnspan=12,sticky=(S,W,N),padx=10,pady=10)
operatorK.grid(row=3,column=0,columnspan=3,sticky=(E,S,N),padx=10)
operatorKCount.grid(row=3,column=3,columnspan=9,sticky=(S,W,N),padx=10)
operatorValuesK.grid(row=3,column=12,columnspan=12,sticky=(W,S,N),padx=10)
operatorB.grid(row=4,column=0,columnspan=3,sticky=(E,S,N),padx=10)
operatorBCount.grid(row=4,column=3,columnspan=9,sticky=(S,W,N),padx=10)
operatorValuesB.grid(row=4,column=12,columnspan=12,sticky=(W,S,N),padx=10)
operatorP.grid(row=5,column=0,columnspan=3,sticky=(E,S,N),padx=10)
operatorPCount.grid(row=5,column=3,columnspan=9,sticky=(S,W,N),padx=10)
operatorValuesP.grid(row=5,column=12,columnspan=12,sticky=(W,S,N),padx=10)
operatorH.grid(row=6,column=0,columnspan=3,sticky=(E,S,N),padx=10)
operatorHCount.grid(row=6,column=3,columnspan=9,sticky=(S,W,N),padx=10)
operatorValuesH.grid(row=6,column=12,columnspan=12,sticky=(W,S,N),padx=10)
operatorD.grid(row=7,column=0,columnspan=3,sticky=(E,S,N),padx=10)
operatorDCount.grid(row=7,column=3,columnspan=9,sticky=(S,W,N),padx=10)
operatorValuesD.grid(row=7,column=12,columnspan=12,sticky=(W,S,N),padx=10)
operatorM.grid(row=8,column=0,columnspan=3,sticky=(E,S,N),padx=10)
operatorMCount.grid(row=8,column=3,columnspan=9,sticky=(S,W,N),padx=10)
operatorValuesM.grid(row=8,column=12,columnspan=12,sticky=(W,S,N),padx=10)
operatorR.grid(row=9,column=0,columnspan=3,sticky=(E,S,N),padx=10)
operatorRCount.grid(row=9,column=3,columnspan=9,sticky=(S,W,N),padx=10)
operatorValuesR.grid(row=9,column=12,columnspan=12,sticky=(W,S,N),padx=10)
resetButton.grid(row=10,column=0,columnspan=6,sticky=(E,S,W,N),padx=10,pady=10)
totalTimeTitle.grid(row=10,column=6,columnspan=12,sticky=(S,W,N),padx=10,pady=10)
totalTime.grid(row=10,column=18,columnspan=3,sticky=(S,W,N),pady=10)
totalTimePlus.grid(row=10,column=21,sticky=(E,S,W,N),pady=10)
operatorRCount2.grid(row=10,column=22,sticky=(E,S,W,N),pady=10)
totalTimeR.grid(row=10,column=23,sticky=(E,S,W,N),pady=10)
operatorSequence.after(400,lambda:operatorSequence.insert(END,"M "))
M.set(M.get()+1)
time.set(round(K.get()*0.20+B.get()*0.10+P.get()*1.10+H.get()*0.40+M.get()*1.35,2))

#when clicking on query field
def onQueryFieldClicked(event):
    #if it's first time to click on query field
    global flagFirstClick
    global flagContinuousClick
    global counter
    counter += 1
    global counterClick
    counterClick += 1
    if (counter - counterClick ==0):
        flagContinuousClick=True
    else:
        flagContinuousClick=False
        counterClick=counter
    if (flagFirstClick):
        operatorSequence.insert(END,"H P B B\n\n")
        H.set(H.get()+1)
        P.set(P.get()+1)
        B.set(B.get()+2)
        flagFirstClick=False
    else:
        #if click on query field continuously
        if (flagContinuousClick):
            operatorSequence.insert(END,"B B\n\n")
            B.set(B.get()+2)
        #if click on query field uncontinuously
        else:
            operatorSequence.insert(END,"\n\nM H B B\n\n")
            H.set(H.get()+1)
            M.set(M.get()+1)
            B.set(B.get()+2)
    operatorSequence.see(END)
    time.set(round(K.get()*0.20+B.get()*0.10+P.get()*1.10+H.get()*0.40+M.get()*1.35,2))

#when typing in query field
def onQueryFieldInput(event):
    global flagContinuouInput
    global counter
    counter += 1
    global counterInput
    counterInput += 1
    if (counter - counterInput == 0):
        flagContinuouInput=True
    else:
        flagContinuouInput=False
        counterInput=counter
    #if keyboard input in query field continuously
    if (flagContinuouInput):
        operatorSequence.insert(END,"K ")
        K.set(K.get()+1)
    else:
        operatorSequence.insert(END,"M H K ")
        M.set(M.get()+1)
        H.set(H.get()+1)
        K.set(K.get()+1)
    operatorSequence.see(END)
    time.set(round(K.get()*0.20+B.get()*0.10+P.get()*1.10+H.get()*0.40+M.get()*1.35,2))

#when clicking on search button to search
def onSearchClicked(event):
    entryText=queryField.get()
    entryLength=len(entryText)
    global flagContinuousClick
    global counter
    counter += 1
    global counterClick
    counterClick += 1
    if (counter - counterClick == 0):
        flagContinuousClick=True
    else:
        flagContinuousClick=False
        counterClick=counter
    if (flagContinuousClick):
        operatorSequence.insert(END,"M P B B R\n\n")
        M.set(M.get()+1)
        P.set(P.get()+1)
        B.set(B.get()+2)
        R.set(R.get()+1)
    else:
        operatorSequence.insert(END,"M H P B B R\n\n")
        M.set(M.get()+1)
        H.set(H.get()+1)
        P.set(P.get()+1)
        B.set(B.get()+2)
        R.set(R.get()+1)
    placeholder.lift(cover)
    queryField.config(state="disabled")
    # operatorSequence.config(state="disabled")
    queryField.unbind('<Key>')
    operatorSequence.see(END)
    time.set(round(K.get()*0.20+B.get()*0.10+P.get()*1.10+H.get()*0.40+M.get()*1.35,2))
    queryField.unbind('<Button-1>')
    queryField.unbind('<Return>')
    searchButton.unbind('<Button-1>')

#when press <ENTER> key to search
def onSearchEnterClicked(event):
    global flagContinuousClick
    global counter
    counter += 1
    global counterInput
    counterInput += 1
    if (counter - counterInput == 0):
        flagContinuousInput=True
    else:
        flagContinuousInput=False
        counterInput=counter
    if (flagContinuousInput):
        operatorSequence.insert(END,"\n\nK R\n\n")
        K.set(K.get()+1)
        R.set(R.get()+1)
    else:
        operatorSequence.insert(END,"M H K R\n\n")
        M.set(M.get()+1)
        H.set(H.get()+1)
        K.set(K.get()+1)
        R.set(R.get()+1)
    placeholder.lift(cover)
    queryField.config(state="disabled")
    # operatorSequence.config(state="disabled")
    queryField.unbind('<Key>')
    operatorSequence.see(END)
    time.set(round(K.get()*0.20+B.get()*0.10+P.get()*1.10+H.get()*0.40+M.get()*1.35,2))
    queryField.unbind('<Button-1>')
    queryField.unbind('<Return>')
    searchButton.unbind('<Button-1>')

# when manually change operator sequence via text box
def onOpSeqFocused(event):
    global operatorSequenceTmp
    operatorSequenceTmp = operatorSequence.get("1.0", 'end-1c')

def updateOpSeq():
    global operatorSequenceTmp
    if (operatorSequence.get("1.0",'end-1c')!=operatorSequenceTmp):
        operatorSequenceTmp = operatorSequence.get("1.0",'end-1c')
        countOp = Counter(operatorSequenceTmp)
        K.set(countOp['K'])
        B.set(countOp['B'])
        P.set(countOp['P'])
        H.set(countOp['H'])
        D.set(countOp['D'])
        M.set(countOp['M'])
        R.set(countOp['R'])
        operatorSequence.see(END)
        time.set(round(K.get()*0.20+B.get()*0.10+P.get()*1.10+H.get()*0.40+M.get()*1.35,2))

def onOpSeqManuallyChanged(event):
    operatorSequence.after(5,updateOpSeq)

#click on "KLM on Wikipedia" button, jump to webpage
def onWikipediaClicked(event):
    webbrowser.open_new(r"en.wikipedia.org/wiki/Keystroke-level_model")

#when clicking on reset button
def onResetClicked(event):
    queryField.config(state="normal")
    operatorSequence.config(state="normal")
    queryField.bind('<Key>',onQueryFieldInput)
    K.set(0)
    B.set(0)
    P.set(0)
    H.set(0)
    D.set(0)
    M.set(0)
    R.set(0)
    time.set(0)
    queryField.delete(0,'end')
    placeholder.lower(cover)
    operatorSequence.delete('1.0',END)
    operatorSequence.after(400,lambda:operatorSequence.insert(END,"M "))
    M.set(M.get()+1)
    time.set(round(K.get()*0.20+B.get()*0.10+P.get()*1.10+H.get()*0.40+M.get()*1.35,2))
    queryField.bind('<Button-1>',onQueryFieldClicked)
    queryField.bind('<Key>',onQueryFieldInput)
    queryField.bind('<Return>',onSearchEnterClicked)
    searchButton.bind('<Button-1>',onSearchClicked)
    global flagFirstClick
    flagFirstClick=True
    global flagContinuousClick
    flagContinuousClick=False
    global flagContinuouInput
    flagContinuouInput=False
    global counter
    counter=0
    global counterClick
    counterClick=0
    global counterInput
    counterInput=0

wikipediaButton.bind('<Button-1>',onWikipediaClicked)
queryField.bind('<Button-1>',onQueryFieldClicked)
queryField.bind('<Key>',onQueryFieldInput)
queryField.bind('<Return>',onSearchEnterClicked)
searchButton.bind('<Button-1>',onSearchClicked)
operatorSequence.bind('<Button-1>', onOpSeqFocused)
operatorSequence.bind('<Key>', onOpSeqManuallyChanged)
resetButton.bind('<Button-1>',onResetClicked)

root.mainloop()
